<a href="https://colab.research.google.com/github/haimin777/IbPy_SMA/blob/master/breast_density.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [10]:
#from google.colab import files
import os
import shutil as sh

!pip install pydicom
import pydicom as pyd
import glob
import tensorflow as tf

import asyncio
import glob
import os
import sys
from os.path import join

import cv2
#import my_density_model_tf
import numpy as np
import pydicom as pyd
from keras.models import load_model
from tqdm import tqdm_notebook

ROOT_DIR = 'drive/My Drive/M-project/X-dense'

In [3]:
if os.path.exists('breast_density_classifier'):
  sh.rmtree('breast_density_classifier')
  
!git clone https://github.com/haimin777/breast_density_classifier.git
#os.chdir('breast_density_classifier')  
#os.chdir('/content/drive')

Cloning into 'breast_density_classifier'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 56 (delta 1), reused 0 (delta 0), pack-reused 50
Unpacking objects: 100% (56/56), done.


In [0]:
import breast_density_classifier
#sys.path.append(os.path.join(ROOT_DIR, 'breast_density_classifier'))  # To find local version of the library
import breast_density_classifier.utils as utils
import breast_density_classifier.density_model as density_model
import breast_density_classifier.layers_tf as layers_tf
import breast_density_classifier.models_tf as models
import breast_density_classifier.density_model_tf as d_models


In [5]:
    parameters = dict(
        device_type='gpu',
        gpu_number=1,
        input_size=(2600, 2000),
        image_path='images/',
        
    )
  
session_config = tf.ConfigProto()
session_config.gpu_options.visible_device_list = str(parameters['gpu_number'])  
x_L_CC = tf.placeholder(tf.float32,
               shape=[None, parameters['input_size'][0], parameters['input_size'][1], 1])
x_R_CC = tf.placeholder(tf.float32,
              shape=[None, parameters['input_size'][0], parameters['input_size'][1], 1])
x_L_MLO = tf.placeholder(tf.float32,
                          shape=[None, parameters['input_size'][0], parameters['input_size'][1], 1])
x_R_MLO = tf.placeholder(tf.float32,
                         shape=[None, parameters['input_size'][0], parameters['input_size'][1], 1])
x = (x_L_CC, x_R_CC, x_L_MLO, x_R_MLO)  
my_model = models.baseline(x, parameters, nodropout_probability=.99)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
!ls breast_density_classifier/saved_models/BreastDensity_BaselineBreastModel/

model.ckpt.data-00000-of-00001	model.ckpt.index  model.ckpt.meta  model.p


In [0]:
model = 'histogram'
#model = 'cnn'
 
parameters = dict(
device_type = 'cpu',
gpu_number = 1,
input_size = (2600, 2000),
image_path = 'images/'
	)
    
if model == 'histogram':    
		parameters['model_class'] = models.BaselineHistogramModel
		parameters['bins_histogram'] = 50
		parameters['initial_parameters'] = 'saved_models/BreastDensity_BaselineHistogramModel/model.ckpt'
        
elif model == 'cnn':
		parameters['model_class'] = models.BaselineBreastModel 
		parameters['initial_parameters'] = 'saved_models/BreastDensity_BaselineBreastModel/model.ckpt'
		
density_model.training(parameters, model)


In [6]:
#try to make new model

from keras.models import Sequential
from keras.layers import InputLayer, Dense, Activation, Conv2D, BatchNormalization, Dropout, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Flatten, AveragePooling2D
from keras.models import Model


config1 = {'activation': 'softmax',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'normal',
   'mode': 'fan_in',
   'scale': 2.0,
   'seed': None}},
 'kernel_regularizer': {'class_name': 'L1L2',
  'config': {'l1': 0.0, 'l2': 0.009999999776482582}},
 'name': 'dense_1',
 'trainable': True,
 'units': 4,
 'use_bias': True}

Using TensorFlow backend.


In [0]:
from keras.layers import Input, Dense, Conv2D, Add, MaxPool2D, Flatten, Concatenate, AvgPool2D, Dropout


In [0]:
a1 = Input(shape=(2600, 2000, 1))
a2 = Input(shape=(2600, 2000, 1))
a3 = Input(shape=(2600, 2000, 1))
a4 = Input(shape=(2600, 2000, 1))

def add_conv(a1, a2, a3, a4, filt_num=32, strides=(2,2)):
  out1 = Conv2D(filters=filt_num, kernel_size=(3,3), strides=strides)(a1)
  out2 = Conv2D(filters=filt_num, kernel_size=(3,3), strides=strides)(a2)
  out3 = Conv2D(filters=filt_num, kernel_size=(3,3), strides=strides)(a3)
  out4 = Conv2D(filters=filt_num, kernel_size=(3,3), strides=strides)(a4)
  
  h = (out1, out2, out3, out4)
  return h

def add_max_pool(inp_layer, strides=(3,3)):
  out1 = MaxPool2D(strides=strides)(inp_layer[0])
  out2 = MaxPool2D(strides=strides)(inp_layer[1])
  out3 = MaxPool2D(strides=strides)(inp_layer[2])
  out4 = MaxPool2D(strides=strides)(inp_layer[3])

  
  h = (out1, out2, out3, out4)
  return h

def add_avg_pooling(inp_layer):
  input_layer_shape = inp_layer[0].get_shape()
  pooling_shape =  (2,2)#[1, input_layer_shape[1], input_layer_shape[2], 1]

  out1 = AvgPool2D(pool_size=pooling_shape, strides=pooling_shape)(inp_layer[0])
  out2 = AvgPool2D(pool_size=pooling_shape, strides=pooling_shape)(inp_layer[1])
  out3 = AvgPool2D(pool_size=pooling_shape, strides=pooling_shape)(inp_layer[2])
  out4 = AvgPool2D(pool_size=pooling_shape, strides=pooling_shape)(inp_layer[3])

  h = (out1, out2, out3, out4)
  return h

def add_flatten(inp_layer):
  out1 = Flatten()(inp_layer[0])
  out2 = Flatten()(inp_layer[1])
  out3 = Flatten()(inp_layer[2])
  out4 = Flatten()(inp_layer[3])
  
  h = Concatenate(axis=1)([out1, out2, out3, out4])
  return h



# 1 conv
h = add_conv(a1, a2, a3, a4)

# 2 conv

h = add_max_pool(h)
h = add_conv(h[0], h[1], h[2], h[3], filt_num=64)
h = add_conv(h[0], h[1], h[2], h[3], filt_num=64, strides=(1,1)) 
h = add_conv(h[0], h[1], h[2], h[3], filt_num=64, strides=(1,1)) 

# 3 conv

h = add_max_pool(h, strides=(2,2))
h = add_conv(h[0], h[1], h[2], h[3], filt_num=128, strides=(1,1)) 
h = add_conv(h[0], h[1], h[2], h[3], filt_num=128, strides=(1,1)) 
h = add_conv(h[0], h[1], h[2], h[3], filt_num=128, strides=(1,1)) 

# 4 conv

h = add_max_pool(h, strides=(2,2))
h = add_conv(h[0], h[1], h[2], h[3], filt_num=128, strides=(1,1)) 
h = add_conv(h[0], h[1], h[2], h[3], filt_num=128, strides=(1,1)) 
h = add_conv(h[0], h[1], h[2], h[3], filt_num=128, strides=(1,1)) 

# 5 conv

h = add_max_pool(h, strides=(2,2))
h = add_conv(h[0], h[1], h[2], h[3], filt_num=256, strides=(1,1)) 
h = add_conv(h[0], h[1], h[2], h[3], filt_num=256, strides=(1,1)) 
h = add_conv(h[0], h[1], h[2], h[3], filt_num=256, strides=(1,1)) 


h = add_avg_pooling(h)
h = add_flatten(h)

#h = Dense(4*256,activation='relu')(h) 
h = Dropout(.15)(h)
out = Dense(4,activation='softmax')(h) 

my_model = Model(inputs=[a1,a2,a3, a4], outputs=out)


In [47]:
model = Sequential()
#model.add(InputLayer((2600, 2000, 1)))
model.add(inp_model([a1, a2, a3]))
model.add(Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), name='conv1'))
#second conv
model.add(MaxPooling2D(strides=(3,3)))
model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(2,2), name='conv2a'))
model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), name='conv2b'))
model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), name='conv2c'))

#3 conv
model.add(MaxPooling2D(strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), name='conv3a'))
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), name='conv3b'))
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), name='conv3c'))

# 4 conv
model.add(MaxPooling2D(strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), name='conv4a'))
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), name='conv4b'))
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), name='conv4c'))

# 5 conv

model.add(MaxPooling2D(strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), name='conv5a'))
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), name='conv5b'))
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), name='conv5c'))
model.add(Activation('softmax'))
model.add(Dropout(.97))


model.add(AveragePooling2D())
#model.add(MaxPooling2D())
model.add(Flatten())

#model.add(Dense(4*256))
model.add(Dense.from_config(config1))

model.summary()





TypeError: ignored

In [0]:
inp_model = Model(inputs=[InputLayer((2600, 2000)), InputLayer(2600, 2000)], outputs=)

TypeError: ignored

In [0]:


def isdicom(file_path):
    try:
        if pyd.dcmread(file_path).ImageType[0] != 'ORIGINAL':
            return True
        else:
            print('RAW image acepted')
            return False
    except Exception as e:
        print(e)
        return False
        
        

def load_dcm_images(image_path):
    """
    Function that loads and preprocess input images
    :param image_path: base path to image
    :param view: L-CC / R-CC / L-MLO / R-MLO
    :return: Batch x Height x Width x Channels array
    """
    ds = pyd.dcmread(image_path)
    #view = 
    image = ds.pixel_array
    
    image = cv2.resize(image, (2000, 2600), interpolation=cv2.INTER_AREA)

    image = image.astype(np.float32)
    utils.normalize_single_image(image)
    image = np.expand_dims(image, axis=0)
    image = np.expand_dims(image, axis=3)

    return image
        

def inference(parameters, verbose=True):
    tf.set_random_seed(7)

    with tf.Graph().as_default():
        with tf.device('/' + parameters['device_type']):
            # initialize input holders
            if parameters["model_type"] == 'cnn':
                x_l_cc = tf.placeholder(tf.float32,
                                        shape=[None, parameters['input_size'][0], parameters['input_size'][1], 1])
                x_r_cc = tf.placeholder(tf.float32,
                                        shape=[None, parameters['input_size'][0], parameters['input_size'][1], 1])
                x_l_mlo = tf.placeholder(tf.float32,
                                         shape=[None, parameters['input_size'][0], parameters['input_size'][1], 1])
                x_r_mlo = tf.placeholder(tf.float32,
                                         shape=[None, parameters['input_size'][0], parameters['input_size'][1], 1])
                x = (x_l_cc, x_r_cc, x_l_mlo, x_r_mlo)
                model_class = models.BaselineBreastModel
            elif parameters["model_type"] == 'histogram':
                x = tf.placeholder(tf.float32, shape=[None, parameters['bins_histogram'] * 4])
                model_class = models.BaselineHistogramModel
            else:
                raise RuntimeError(parameters["model_type"])

            # holders for dropout and Gaussian noise
            nodropout_probability = tf.placeholder(tf.float32, shape=())
            gaussian_noise_std = tf.placeholder(tf.float32, shape=())

            # construct models
            model = model_class(parameters, x, nodropout_probability, gaussian_noise_std)
            y_prediction_density = model.y_prediction_density

        # allocate computation resources
        if parameters['device_type'] == 'gpu':
            session_config = tf.ConfigProto()
            session_config.gpu_options.visible_device_list = str(parameters['gpu_number'])
        elif parameters['device_type'] == 'cpu':
            session_config = tf.ConfigProto(device_count={'GPU': 0})
        else:
            raise RuntimeError(parameters['device_type'])

        with tf.Session(config=session_config) as session:
            session.run(tf.global_variables_initializer())

            # loads the pre-trained parameters if it's provided
            density_model_tf.optimistic_restore(session, parameters['model_path'])

            # load input images

            dcm_paths = [os.path.join(parameters['image_path'], dcm_file) for dcm_file in os.listdir(parameters['image_path']) if
                         isdicom(os.path.join(parameters['image_path'], dcm_file))]

            datum_l_cc = load_dcm_images(dcm_paths[0])
            datum_r_cc = load_dcm_images(dcm_paths[1])
            datum_l_mlo = load_dcm_images(dcm_paths[2])
            datum_r_mlo = load_dcm_images(dcm_paths[3])

            # populate feed_dict for TF session
            # No dropout and no gaussian noise in inference
            feed_dict_by_model = {nodropout_probability: 1.0, gaussian_noise_std: 0.0}
            if parameters["model_type"] == 'cnn':
                feed_dict_by_model[x_l_cc] = datum_l_cc
                feed_dict_by_model[x_r_cc] = datum_r_cc
                feed_dict_by_model[x_l_mlo] = datum_l_mlo
                feed_dict_by_model[x_r_mlo] = datum_r_mlo
            elif parameters["model_type"] == 'histogram':
                feed_dict_by_model[x] = utils.histogram_features_generator(
                    #[datum_l_cc, datum_r_cc, datum_l_mlo, datum_r_mlo],
                    [datum_r_mlo, datum_l_cc, datum_l_mlo,  datum_r_cc],

                    parameters,
                )

            # run the session for a prediction
            prediction_density = session.run(y_prediction_density, feed_dict=feed_dict_by_model)

            if verbose:
                # nicely prints out the predictions
                print('Density prediction:\n' +
                      '\tAlmost entirely fatty (0):\t\t\t' + str(prediction_density[0, 0]) + '\n' +
                      '\tScattered areas of fibroglandular density (1):\t' + str(prediction_density[0, 1]) + '\n' +
                      '\tHeterogeneously dense (2):\t\t\t' + str(prediction_density[0, 2]) + '\n' +
                      '\tExtremely dense (3):\t\t\t\t' + str(prediction_density[0, 3]) + '\n')

            return prediction_density[0]


In [0]:
os.chdir('/content/breast_density_classifier')

In [0]:
parameters = {
        'model_type': 'histogram',
        'bins_histogram': 50,
        'model_path': "/content/drive/My Drive/M-project/saved_models/BreastDensity_BaselineHistogramModel/model.ckpt",
        'device_type': "cpu",
        'gpu_number': 0,
        # 'image_path': '/home/haimin/PycharmProjects/xray_density/breast_density_classifier/dcm_images'
        # 'image_path': glob.glob(folder+'/FD.*/MAMMOGRAM..Screening*')[0]
        'image_path': None
              }

folders = glob.glob('/content/drive/My Drive/M-project/ACR_1/*')
print(folders[0])
#inference(parameters, verbose=True)

for folder in folders:
  parameters['image_path'] = folder
  inference(parameters, verbose=True)

'''
for folder in folders:
  parameters['image_path'] = folder
  try:
    inference(parameters, verbose=True)
    
  except Exception as e:
    print(e)
''' 

In [0]:
pyd.dcmread(join(folders[1], '20180404142657.104.1978749.dcm'))

(0008, 0008) Image Type                          CS: ['DERIVED', 'PRIMARY', '', 'RIGHT']
(0008, 0016) SOP Class UID                       UI: Secondary Capture Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.826.0.1.3680043.2.1143.5048753967654858577015913393542830299
(0008, 0020) Study Date                          DA: '20190118'
(0008, 0023) Content Date                        DA: '20180404'
(0008, 0030) Study Time                          TM: '235915'
(0008, 0033) Content Time                        TM: '132701'
(0008, 0060) Modality                            CS: 'MG'
(0020, 000d) Study Instance UID                  UI: 1.2.826.0.1.3680043.2.1143.6431022368119457168403934632901395813
(0020, 000e) Series Instance UID                 UI: 1.2.826.0.1.3680043.2.1143.9523496642559791133477039714320579440
(0028, 0002) Samples per Pixel                   US: 1
(0028, 0004) Photometric Interpretation          CS: 'MONOCHROME2'
(0028, 0010) Rows                       

In [0]:
parameters = {
        'model_type': 'histogram',
        'bins_histogram': 50,
        'model_path': "/content/drive/My Drive/M-project/saved_models/BreastDensity_BaselineHistogramModel/model.ckpt",
        'device_type': "cpu",
        'gpu_number': 0,
        # 'image_path': '/home/haimin/PycharmProjects/xray_density/breast_density_classifier/dcm_images'
        # 'image_path': glob.glob(folder+'/FD.*/MAMMOGRAM..Screening*')[0]
        'image_path': '/content/drive/My Drive/M-project/ACR_1/63873'
              }

inference(parameters, verbose=True)

    
    

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/M-project/saved_models/BreastDensity_BaselineHistogramModel/model.ckpt
Density prediction:
	Almost entirely fatty (0):			0.17023645
	Scattered areas of fibroglandular density (1):	0.78430045
	Heterogeneously dense (2):			0.042454615
	Extremely dense (3):				0.0030084394



array([0.17023645, 0.78430045, 0.04245462, 0.00300844], dtype=float32)